In [1]:
%pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import sys
import boto3

In [3]:
client = boto3.client("bedrock-runtime")

In [4]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

sales.csv has been created!


In [5]:
prompt_data = """
Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- Identify the top 5 best selling products by total sales for the year 2023
- Calculate the monthly average sales for the year 2023

Assistant:
"""

In [6]:
body = json.dumps({
    "inputText": prompt_data,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.5,
        "topP": 0.9
    }
})

In [7]:
resp = client.invoke_model(
    body=body, 
    modelId="amazon.titan-tg1-large", 
    accept='application/json', 
    contentType='application/json'
)

In [8]:
body = json.loads(resp.get('body').read())

In [9]:
for result in body['results']:
    print(f"Token count: {result['tokenCount']}")
    print(f"Output text: {result['outputText']}")
    print(f"Completion reason: {result['completionReason']}")

Token count: 160
Output text: The top 5 best selling products by total sales for the year 2023 can be identified using the following SQL query:

```SQL
SELECT product_id
FROM sales_data
WHERE YEAR = 2023
GROUP BY product_id
ORDER BY SUM(units_sold) DESC
LIMIT 5;
```

The monthly average sales for the year 2023 can be calculated using the following SQL query:

```SQL
SELECT YEAR, MONTH, AVG(units_sold)
FROM sales_data
WHERE YEAR = 2023
GROUP BY YEAR, MONTH;
```

This query will provide the average sales for each month of the year 2023.
Completion reason: FINISH
